In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
train_features, test_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv'), pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
# Sample submission file
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

# Target data
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [ ]:
import tensorflow as tf 
from tensorflow.keras import models ,Model , Input
from tensorflow.keras.utils import plot_model 
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.initializers import TruncatedNormal, he_uniform, he_normal
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D, ReLU, Dropout,Flatten, Dense, InputLayer, Concatenate, Add, SeparableConv2D, Layer

In [ ]:
print(train_features.shape)
print(test_features.shape)
print(train_targets.shape)

In [ ]:
indices = train_features[train_features['cp_type'] == 'trt_cp'].index.to_list()

In [ ]:
train_targets = train_targets.iloc[indices, :].reset_index(drop=True)
train_targets_nonscored = train_targets_nonscored.iloc[indices, :].reset_index(drop=True)
train_features = train_features.iloc[indices, :].reset_index(drop=True)

In [ ]:
train_targets

In [ ]:
train_targets_nonscored

In [ ]:
cp_dose = {'D1': 1, 'D2': -1}

In [ ]:
train_features['cp_dose'], test_features['cp_dose'] = train_features['cp_dose'].map(cp_dose), test_features['cp_dose'].map(cp_dose)

In [ ]:
del train_features['sig_id']
del train_features['cp_type']
del test_features['sig_id']
del test_features['cp_type']

In [ ]:
train_features

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_len = train_features.shape[0]
comb = pd.concat([train_features, test_features])

In [ ]:
genes_comb = comb.iloc[:,2:774]
cells_comb = comb.iloc[:,774:]

In [ ]:
pca_genes = PCA(n_components=20).fit_transform(genes_comb)
pca_cells = PCA(n_components=10).fit_transform(cells_comb)
pca_all = PCA(n_components=100).fit_transform(comb)

In [ ]:
comb_scaled = MinMaxScaler().fit_transform(comb)

In [ ]:
scaled_train_features = comb_scaled[0:train_len]
scaled_test_features = comb_scaled[train_len:]

In [ ]:
X = train_features.astype('float64').copy()
Xs = pd.DataFrame(scaled_train_features.astype('float64')).copy()

In [ ]:
pca_all_train = pca_all[:train_len]
pca_cells_train = pca_cells[:train_len]
pca_genes_train = pca_genes[:train_len]

In [ ]:
X_test = test_features.astype('float64').copy()
Xs_test = pd.DataFrame(scaled_test_features.astype('float64')).copy()

In [ ]:
pca_all_test = pca_all[train_len:]
pca_cells_test = pca_cells[train_len:]
pca_genes_test = pca_genes[train_len:]

In [ ]:
y = train_targets.iloc[:,1:].astype('float64').copy()
y_non = train_targets_nonscored.iloc[:,1:].astype('float64').copy()

In [ ]:
y_all = pd.concat([train_targets, y_non], axis=1).iloc[:,1:].astype('float64').copy()

In [ ]:
Cells = X.iloc[:,774:].copy()
Cells_scaled = Xs.iloc[:,774:].copy() 
Genes = X.iloc[:,3:774].copy()
Genes_scaled = Xs.iloc[:,3:774].copy()
Cells_image = Cells.values.reshape(Cells.shape[0], 10, 10, 1)
Cells_image_scaled = Cells_scaled.values.reshape(Cells.shape[0], 10, 10, 1)

In [ ]:
Cells_test = X_test.iloc[:,774: ].copy()
Cells_test_scaled = Xs_test.iloc[:,774: ].copy()
Genes_test = X_test.iloc[:,3:774].copy()
Genes_test_scaled = Xs_test.iloc[:,3:774].copy()
Cells_image_test = Cells_test.values.reshape(Cells_test.shape[0], 10, 10, 1)
Cells_image_test_scaled = Cells_test_scaled.values.reshape(Cells_test.shape[0], 10, 10, 1)

In [ ]:
all_input = Input(shape=(X.shape[1]), name='All')
cells_input = Input(shape=(Cells.shape[1]), name='Cells')
genes_input = Input(shape=(Genes.shape[1]), name='Genes')
cell_image_input = Input(shape=(10,10,1), name='Cells_image')
pca_cell_input = Input(shape=pca_cells.shape[1], name='PCA_cells')
pca_gene_input = Input(shape=pca_genes.shape[1], name='PCA_genes')
pca_all_input = Input(shape=pca_all.shape[1], name='PCA_all')

In [ ]:
def residual_module(x, add, std=1, Unit=100, seed=11):
    """ A residual connection module """
    out = Dense(Unit, activation='relu', kernel_initializer=TruncatedNormal(0, std, seed))(x)
    out = BatchNormalization()(out)
    out = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, std, seed))(out)
    
    return Add()([out, add])

In [ ]:
def linear_NN(x, std, seed=1, unit=32):
    c = Dense(unit, activation='selu', kernel_initializer=TruncatedNormal(0, std, seed))(x)
    c = Dense(unit * 2, kernel_initializer=TruncatedNormal(0, std, seed))(c)
    c = BatchNormalization()(c)
    c = Dense(unit * 4, kernel_initializer=TruncatedNormal(0, std, seed))(c)
    c = BatchNormalization()(c)
    c = Dense(unit * 8, kernel_initializer=TruncatedNormal(0, std, seed))(c)
    
    return c
    
cells_output = linear_NN(cells_input, 1e-1, unit=16)
p_cells = BatchNormalization()(cells_output)
p_cells = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 5, 1))(p_cells)

genes_output = linear_NN(genes_input, 1.5e-1, unit=16)
p_genes = BatchNormalization()(genes_output)
p_genes = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 10, 1))(p_genes)

all_output = linear_NN(all_input, 5e-1, unit=64)
p_all = BatchNormalization()(all_output)
p_all = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 8, 1))(p_all)

pca_cell_output = linear_NN(pca_cell_input, 1, seed=2, unit=16)
p_pca_cell = BatchNormalization()(pca_cell_output)
p_pca_cell = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 4, 123))(p_pca_cell)

pca_gene_output = linear_NN(pca_gene_input, 1, seed=21)
p_pca_gene = BatchNormalization()(pca_gene_output)
p_pca_gene = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 11, 123))(p_all)

pca_all_output = linear_NN(pca_all_input, 1, seed=31, unit=64)
p_pca_all = BatchNormalization()(pca_all_output)
p_pca_all = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(0, 9, 123))(p_pca_all)


comb = Concatenate()([cells_output, genes_output, all_output, pca_cell_output, pca_gene_output, pca_all_output])
comb = Dense(512, kernel_initializer=TruncatedNormal(0, 5, 13))(comb)
comb = BatchNormalization()(comb)

out = residual_module(comb, p_cells, std=5, Unit=100, seed=1)
    
out = residual_module(out, p_genes, std=5, Unit=100, seed=3)

out = residual_module(out, p_all, std=5, Unit=100, seed=5)

out = residual_module(out, p_pca_cell, std=5, Unit=100, seed=7)
    
out = residual_module(out, p_pca_gene, std=5, Unit=100, seed=9)

out = residual_module(out, p_pca_all, std=5, Unit=100, seed=11)

out = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(10, 20, 11))(out)

out = Add()([out, p_all, p_cells, p_genes, p_pca_all, p_pca_gene, p_pca_cell])

out = Dense(y_all.shape[1], activation='sigmoid', kernel_initializer=TruncatedNormal(5, 25, 11))(out)

model = Model(inputs=[cells_input, genes_input, all_input, pca_cell_input, pca_gene_input, pca_all_input], outputs=[out])

In [ ]:
model.count_params() / 1e6

In [ ]:
plot_model(model, show_shapes=1, show_layer_names=0)

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy

In [ ]:
def scheduler(epoch, lr): return max(1e-25, lr * 0.96 ** (epoch // 50))

model.compile(optimizer=Adam(0.0015), loss=BinaryCrossentropy())
history = model.fit(
        x=[Cells_scaled, Genes_scaled, Xs, pca_cells_train, pca_genes_train, pca_all_train], 
        y=y_all, epochs=10000, batch_size=128, validation_split=0.2, verbose=0,
        callbacks=[
#             LearningRateScheduler(scheduler), 
            ReduceLROnPlateau(factor=0.95, patience=20, verbose=True, monitor='val_loss', min_lr=1e-45, min_delta=0),
            EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True, min_delta=0, verbose=True)
        ],
        shuffle=True
    )

In [ ]:
prediction = model.predict([Cells_test_scaled, Genes_test_scaled, Xs_test, pca_cells_test, pca_genes_test, pca_all_test])[:,:y.shape[1]]
# prediction[prediction > 0.5] = 1 
# prediction[prediction < 1e-8] = 0
submission.iloc[:, 1:] = prediction

In [ ]:
submission.to_csv('/kaggle/working/submission_MOA2.csv', index=False)